# FLIP(01):  Advanced Data Science
**(Module 00: Matrix Analysis)**

---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use, but NOT allowed to change or distribute this package.

Prepared by and for 
**Student Members** |
2006-2018 [TULIP Lab](http://www.tulip.org.au)

---


# Session 00 - Special Matrix 


### Several special matries

 Nomral Matrix

 Symmetric Matrix(in real space)/ Hermite Matrix(in complex space)

 Orthgonal Matrix/ Unitary Matrix

 Hadamard Matrix

 Simple Matrix

 Householder Matrix

In [ ]:
import numpy as np
from scipy import linalg as la




def isNormal(A, method = 'definition'):

    # use Schur inequality to determine whether it's normal
    if method == 'Schur':
        # initialize eigenValue
        eigenValue = la.eig(A)[0]

        if abs(np.sum(eigenValue**2) - la.norm(A, 'fro')**2) < 0.00001:
            return True
        else:
            return False
    # use definition
    else:
        if abs((A.conjugate().T.dot(A) - A.dot(A.conjugate().T)).all()) < 0.00001:
            return True
        else:
            return False


def isSymmetric(A):

    if abs((A.T - A).any()) > 0.00001:
        return False
    else:
        return True

def isHermite(A):

    if abs((A.conjugate().T - A).any()) > 0.00001:
        return False
    else:
        return True

def isOrthogonal(A):

    if abs((A.T.dot(A) - np.eye(A.shape[0])).all()) < 0.00001:
        return True
    else:
        return False

def isUnitary(A):

    if abs((A.conjugate().T.dot(A) - np.eye(A.shape[0])).all()) < 0.00001:
        return True
    else:
        return False

def isSimple(A):

    # check if A is a squre matrix
    if A.shape[1] != A.shape[0]:
        return False

    eigenValues, eigenVectors = la.eig(A)


    while (eigenValues.shape[0] != 0):

        #dictValues.update({eigenValues[0]: 1})

        index = np.argwhere(abs(eigenValues - eigenValues[0]) < 0.00001)
        algebraicMulti = len(index)

        geometricMulti = eigenVectors[:, index].shape[1]

        if algebraicMulti != geometricMulti:
            return False

        #dictValues.update({eigenValues[0]: len})
        eigenValues = np.delete(eigenValues, index)

    # stack another spaces of eigenvalue and eigenvector

    return True


''' Constrct a householder matrix/transformation
    v is the direction vector, not neccessary to be unit
    the function householder(v) would turn non-unit v into a unit vector
'''

'''Properties of Householder
    1, is hermitian
    2, is Unitary
    3, is involutory: H^2 = I
    4, has eigenvalues +1 and -1
    5, its determinant is -1 (product of eigenvalues)
'''
def householder(v):

    # check if vector v's 2-norm is 1
    # If not, make it unit
    if la.norm(v, 2) != 1:
        v = v/la.norm(v, 2)

    return np.eye(np.outer(v,v.conjugate().T).shape[0]) - 2 * np.outer(v, v.T.conjugate().T)


# Given a vector x, return a unit vector v and a scalar beta that form a householder transformation which projects x onto basis e1
def householder_vector(x):

    dimensionX  = len(x)
    sigma = x[1:].conjugate().T.dot(x[1:])
    v = np.vstack((1, x[1:]))

    if sigma == 0:
        beta = 0
        return v, beta
    else:
        miu = np.sqrt(x[0]**2 / sigma)
        if x[0] <= 0:
            v[0] = x[0] - miu
        else:
            v[0] = - sigma / (x[0] + miu)
        beta = 2 * v[0]**2 / (sigma + v[0]**2)
        v = v / la.norm(v, 2)

        return v, beta

# a test fuction that asks whether a particular matrix is one of the special matries above
def testIS():

    # A is a symmetric matrix, and thereby is normal
    A = np.array([[1,4,6],
                  [4,5,7],
                  [6,7,9]])

    # B is a complex matrix
    B = np.array([[1,4+3j,6-2j],
                  [4-3j,5,7-1j],
                  [6+2j,7+1j,9]])

    # C is an orthogonal matrix
    C = np.array([[np.cos(30.),-np.sin(30.)],
                  [np.sin(30.),np.cos(30.)]])

    print('Matrix A: ', A)
    print('Is A a normal Matrix: ', isNormal(A))
    print('Use Schur inequality to determine its normality: ', isNormal(A, 'Schur'))
    print('Is A a symmetric Matrix: ', isSymmetric(A),'\n' )
    print('Matrix B: ', B)
    print('Is B a Hermite Matrix: ', isHermite(B),'\n')
    print('Matrix C: ', C)
    print('Is C an orthogonal Matrix: ', isOrthogonal(C), '\n')


def test():

    v = np.array([0.5, 1, 4, 0.8])
    H = householder(v)
    print('vector v: ', v)
    print('Householder based on v: ', H)
    print('Test the properties of Householder: ')
    print('is it Unitary: ', isUnitary(H))
    print('is it hermitian: ', isHermite(H))

    E = np.array([[4,6,0],
                  [-3,-5,0],
                  [-3,-6,1]])


if __name__ == '__main__':
    testIS()
test()

### Hadamard Matrix

In [ ]:
import numpy as np
from scipy import linalg as la

''' Hadamard matrix can be constructed by calling built-in function of scipy
    la.hadamard(n)
'''

# initialize a hadamard matrix by Sylvester's construction
def hadamard(n, method = 'Sylvester'):
    '''
    initialize a hadamard matrix by Sylvester's construction
    '''

    if method == 'Sylvester':
        H2 = np.array([[1,1],[1,-1]])
        H = np.array([[1,1],[1,-1]])
        if n == 1:
            return np.array([[1]])
        else:
            for i in range(1, n):
                H = la.kron(H2, H)
            return H


def test():

    Hard = hadamard(4)
    print(Hard)

def testIn():

    Hard = la.hadamard(16)
    print(Hard)

if __name__ == '__main__':
    test()

###  Givens rotation of rank 2

In [ ]:
import numpy as np
from scipy import linalg as la

# Givens rotation of rank 2
def givens(x, i=0, j=1):
    '''
    Givens rotation of rank 2
    '''
    if x[j] == 0:
        c, s = 1, 0
    else:
        if abs(x[j]) > abs(x[i]):
            t = -x[i]/x[j]
            s = 1/np.sqrt(1 + t**2)
            c = s * t
        else:
            t = -x[j]/x[i]
            c = 1/np.sqrt(1 + t**2)
            s = c * t

    t1 = x[i]
    t2 = x[j]
    x[i] = c*t1 - s*t2
    x[j] = s*t1 + c*t2

    return c, s, x

# fast givens transformation
def fast_givens(x, d):
    '''
    fast givens transformation
    '''
    if x[1] != 0:
        alpha = -x[0]/x[1]
        beta = -alpha * d[1]/d[0]
        r = -alpha * beta
        if r <= 1:
            ty = 1
            tal = d[0]
            d[0] = (1+r) * d[1]
            d[1] = (1+r) * tal
        else:
            ty = 2
            alpha, beta, r = 1/alpha, 1/beta, 1/r
            d[0] = (1+r) * d[0]
            d[1] = (1+r) * d[1]
    else:
        ty = 2
        alpha, beta = 0, 0

    return alpha, beta, ty



# apply givens rotation to matrix A
def givens_rotation(A, c, s, i, k):
    '''
    apply givens rotation to matrix A
    '''
    for j in range(A.shape[0]):
        t1 = A[j, i]
        t2 = A[j, k]
        A[j, i] = c*t1 - s*t2
        A[j, k] = s*t1 + c*t2


def givens_matrix(c, s):

    return np.array([[c, -s],
                     [s, c]])

def test():

    x = np.array([1, 2, 3, 4])
    print(givens(x, 1, 3)[2])

if __name__ == '__main__':
    test()

In [ ]:
import numpy as np
from scipy import linalg as la


# return a scaled matrix of A
def scaleMatrix(A, scale = [1,1]):
    '''
    return a scaled matrix of A
    '''
    sm = int(A.shape[0] / scale[0])
    sn = int(A.shape[1] / scale[1])

    output = np.empty((sm, sn))

    for i in range(sm):
        for j in range(sn):
            sum = 0
            for si in range(i * scale[0], (i+1) * scale[0]):
                for sj in range(j * scale[1], (j+1) * scale[1]):
                    sum += A[si, sj]

            output[i,j] = sum / (scale[0] * scale[1])

    return output

# rotate matrix by 180 degrees
def rot180(A):
    '''
    rotate matrix by 180 degrees
    '''
    return rot90(rot90(A))

# rotate matrix by 90 degrees
def rot90(A):
    '''
    rotate matrix by 90 degrees
    '''

    m = A.shape[0]
    n = A.shape[1]
    output = np.empty((n, m))

    for i in range(m):
        for j in range(n):

            output[j, m-1-i] = A[i, j]

    A = output
    return A

# rotation, counterclock
def Crot90(A):
    '''
    rotation, counterclock
    '''
    return rot90(rot180(A))

# return the Full convolution between Matrix A and the kernel
def conFull(A, kernel):
    '''
    return the Full convolution between Matrix A and the kernel
    '''
    kernel = rot180(kernel)

    if len(A.shape) != len(kernel.shape):
        raise Exception('Kernel is larger than the matrix', '\n')

    if len(A.shape) == 2:

        m = A.shape[0]
        n = A.shape[1]
        km = kernel.shape[0]
        kn = kernel.shape[1]

        extendMatrix = np.empty((m+2*(km-1), n+2*(kn-1)))
        for i in range(m):
            for j in range(n):
                extendMatrix[i+km-1, j+kn-1] = A[i, j]
        return conValid(extendMatrix, kernel)

    if len(A.shape) == 3:

        m = A.shape[0]
        n = A.shape[1]
        h = A.shape[2]
        km = kernel.shape[0]
        kn = kernel.shape[1]
        kh = kernel.shape[2]

        extendMatrix = np.ones((m+2*(km-1), n+2*(kn-1), h+2*(kh-1)))
        for i in range(m):
            for j in range(n):
                for k in range(h):
                    extendMatrix[i+km-1, j+kn-1, k+kh-1] = A[i, j, k]
        return conValid(extendMatrix, kernel)

    else:
        raise Exception("Under Construction")


# return the valid convolution between Matrix A and the kernel
def conValid(A, kernel):
    '''
    return the valid convolution between Matrix A and the kernel
    '''
    kernel = rot180(kernel)

    if len(A.shape) != len(kernel.shape):
        raise Exception("Kernel is large than the matrix", '\n')

    if len(A.shape) == 2:

        m = A.shape[0]
        n = A.shape[1]
        km = kernel.shape[0]
        kn = kernel.shape[1]

        kms = m - km + 1
        kns = n - kn + 1

        output = np.empty((kms, kns))

        for i in range(kms):
            for j in range(kns):
                sum = 0
                for ki in range(km):
                    for kj in range(kn):
                        sum += A[i+ki, j+kj] * kernel[ki, kj]

                output[i, j] = sum

        return output

    if len(A.shape) == 3:

        m = A.shape[0]
        n = A.shape[1]
        h = A.shape[2]
        km = kernel.shape[0]
        kn = kernel.shape[1]
        kh = kernel.shape[2]

        kms = m - km + 1
        kns = n - kn + 1
        khs = h - kh + 1

        output = np.empty((kms, kns, khs))
        for i in range(kms):
            for j in range(kns):
                for k in range(khs):
                    sum = 0
                    for ki in range(km):
                        for kj in range(kn):
                            for kk in range(kh):
                                sum += A[i+ki, j+kj, k+kh] * kernel[ki, kj, kk]


                    output[i, j, k] = sum

        return output

    if len(A.shape) == 1:
        return np.convolve(A, kernel)

    else:
        raise Exception('Under Construction')

        length = len(A.shape)

        a = np.empty((length))
        for i in range(length):
            a[i] = A.shape[i]

        b = np.empty((length))
        for i in range(length):
            b[i] = kernel.shape[i]

        c = np.empty((length))
        for i in range(length):
            c[i] = a[i] - b[i] + 1

        output = np.empty(c)



#--------------------------------------------------------------

# test whether scaleMatrix() works
def testScale():

    A = np.array([[1,3,5,7,9,10],
                  [2,4,6,8,10,12],
                  [3,5,6,8,11,13],
                  [5,7,9,11,13,15],
                  [1,2,3,4,5,6],
                  [2,3,4,5,6,7]])

    print('A: ', A, '\n', 'Scale of A: ', scaleMatrix(A, (2,2)))

def test():

    A = np.array([[1,3,5,7,9,10],
                  [2,4,6,8,10,12],
                  [3,5,6,8,11,13],
                  [5,7,9,11,13,15],
                  [1,2,3,4,5,6],
                  [2,3,4,5,6,7]])

    kernel = np.array([[1,2,1],
                       [2,1,2]])

    print('Rotate A:\n ', rot90(kernel))
    print("Convolution: \n", conValid(A, kernel))
    print('Full Convolution:\n ', conFull(A, kernel))


if __name__ == '__main__':

    test()